In [49]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

#Load train and test data
train_data=pd.read_csv('C:\\Users\\abhir\\OneDrive\\Desktop\\Data Science using Python\\train.csv',encoding='ISO-8859-1')
test_data=pd.read_csv('C:\\Users\\abhir\\OneDrive\\Desktop\\Data Science using Python\\test.csv')

#Filter null valued compliance rows
train_data=train_data[(train_data['compliance']==1) | (train_data['compliance']==0)]
#load address data
address=pd.read_csv('C:\\Users\\abhir\\OneDrive\\Desktop\\Data Science using Python\\addresses.csv')
#load location data
latlons=pd.read_csv('C:\\Users\\abhir\\OneDrive\\Desktop\\Data Science using Python\\latlons.csv')
address=address.set_index('address').join(latlons.set_index('address'),how='left')
#Join address and location to train and test data
train_data=train_data.set_index('ticket_id').join(address.set_index('ticket_id'))
test_data=test_data.set_index('ticket_id').join(address.set_index('ticket_id'))
#Filter null valued hearing date rows
train_data=train_data[~train_data['hearing_date'].isnull()]
#Remove non-existing features in test data
train_remove_list=['payment_amount','payment_date','payment_status','balance_due','collection_status','compliance_detail']
train_data.drop(train_remove_list,axis=1,inplace=True)
#Remove string data
string_remove_list=['violator_name','zip_code','country','city','inspector_name','violation_street_number','violation_street_name','violation_zip_code','violation_description','mailing_address_str_number','mailing_address_str_name','non_us_str_code','agency_name','state','disposition','ticket_issued_date','hearing_date','grafitti_status','violation_code']
train_data.drop(string_remove_list,axis=1,inplace=True)
test_data.drop(string_remove_list,axis=1,inplace=True)
#Fill NaN Lat/Long Values
train_data.lat.fillna(method='pad',inplace=True)
train_data.lon.fillna(method='pad',inplace=True)
test_data.lat.fillna(method='pad',inplace=True)
test_data.lon.fillna(method='pad',inplace=True)
#Select target parameter and remove it from X_train
y_train=train_data.compliance
X_train=train_data.drop('compliance',axis=1)
X_test=test_data
#Scaling features to reduce computation time
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)


#Build and Train Classifier Model
clf=MLPClassifier(hidden_layer_sizes=[100,10],alpha=0.001,random_state=0,solver='lbfgs',verbose=0)
clf.fit(X_train_scaled,y_train)

#Predict probabilities
y_proba=clf.predict_proba(X_test_scaled)[:,1]
#Integrate with reloaded test data

test_df=pd.read_csv('C:\\Users\\abhir\\OneDrive\\Desktop\\Data Science using Python\\test.csv',encoding='ISO-8859-1')
test_df['compliance']=y_proba
test_df.set_index('ticket_id',inplace=True)
#grid_values={'alpha':[0.001,0.01],'hidden_layer_sizes':[[100,10],[150,10]]}
#grid_clf_auc=GridSearchCV(clf,param_grid=grid_values,scoring='roc_auc')
#grid_clf_auc.fit(X_train_scaled,y_train)

#print('Grid best parameter(max. AUC): ',grid_clf_auc.best_params_)
#print('Grid best score(AUC): ',grid_clf_auc.best_score_)


print(test_df.compliance)









C:\Users\abhir\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ticket_id
284932    0.057541
285362    0.003918
285361    0.084137
285338    0.076204
285346    0.139730
            ...   
376496    0.003620
376497    0.003620
376499    0.105810
376500    0.105792
369851    0.207185
Name: compliance, Length: 61001, dtype: float64


In [31]:
train_data[train_data['lat'].isnull()]


,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,compliance,lat,lon
ticket_id,,,,,,,,,,
89535,50.0,20.0,10.0,5.0,0.0,0.0,85.0,0.0,NaN,NaN
223598,250.0,20.0,10.0,25.0,0.0,0.0,305.0,0.0,NaN,NaN
